# 1. Raw text

In [3]:
db_pw = open('C:/Users/Jake/Documents/Projects/jakes_db.txt').read()

from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres:%s@localhost:5432/disaster' % db_pw)
conn = engine.connect()

In [30]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)

train = pd.read_sql('SELECT * FROM train;', conn)
train.head()

,index,id,keyword,location,text,target
0,0,1,None,None,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,1,4,None,None,Forest fire near La Ronge Sask. Canada,1
2,2,5,None,None,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or...,1
3,3,6,None,None,"13,000 people receive #wildfires evacuation orders in California",1
4,4,7,None,None,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [31]:
train.shape

(7613, 6)

In [32]:
train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [33]:
train = train[['target', 'text']]
train.head()

,target,text
0,1,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
1,1,Forest fire near La Ronge Sask. Canada
2,1,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or...
3,1,"13,000 people receive #wildfires evacuation orders in California"
4,1,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school


# Data Cleaning Function

2. Tokenize
3. Clean Text - stopwords, punctuation, lower case, lemmatize/stem

In [38]:
import string
import re
import nltk

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

def clean_tweets(tweet):
    tweet = ''.join([c.lower() for c in tweet if c not in string.punctuation]) #remove punctuation
    tweet = re.split('\W+', tweet) #tokenize tweet
    tweet = [wn.lemmatize(word) for word in tweet if word not in stopwords and word != ''] #remove stopwords & lemmatize
    return(tweet)

train['clean_tweet'] = train['text'].apply(lambda x: clean_tweets(x))
train.head()

,target,text,clean_tweet
0,1,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,"[deed, reason, earthquake, may, allah, forgive, u]"
1,1,Forest fire near La Ronge Sask. Canada,"[forest, fire, near, la, ronge, sask, canada]"
2,1,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or...,"[resident, asked, shelter, place, notified, officer, evacuation, shelter, place, order, expected]"
3,1,"13,000 people receive #wildfires evacuation orders in California","[13000, people, receive, wildfire, evacuation, order, california]"
4,1,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,"[got, sent, photo, ruby, alaska, smoke, wildfire, pours, school]"


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer=clean_tweets)
train_tfidf = tfidf_vect.fit_transform(train['text'])
print(train_tfidf.shape)
print(tfidf_vect.get_feature_names())

(7613, 20999)
['0', '0011', '001116', '0025', '005225', '010156', '010217', '0104', '010401', '0106', '0111', '012032', '012624', '015025', '02', '0206', '0215', '03', '0306', '0308', '030811', '034', '0400', '045', '05', '05082015', '05th', '06', '0605', '061', '063243', '063424', '06jst', '07', '070', '0700', '0730', '075', '080', '0800', '080215', '0802pm', '080515', '080615', '08062015', '08072015', '08315', '0840728', '09', '0913', '0936', '0day', '0npzp', '1', '10', '100', '1000', '10000', '100000', '1000amp1000', '1008pla', '1008planet', '100bn', '100mb', '100nd', '101', '1015', '1015pm', '1028', '103', '1030', '1030pm', '103424', '1038pm', '1040', '10401', '1040pm', '105', '106', '1061', '1061thetwister', '10638', '107', '1079', '109', '10am', '10k', '10km', '10m', '10monthold', '10news', '10pm', '10th', '10w', '10x', '11', '1100', '11000', '11000000', '110358', '111020', '1115', '112', '113', '1130', '1130a', '1130bst', '11382', '114', '1145', '115', '115film', '118', '119000'

# Vectorize
    - Counts
    - N-gram
    - TF-IDF

5. Machine Learning Algorithm
6. Identify Disasters in Tweets